## Introduction


- Choropleth maps are probably one of the most popular geospatial data visualizations.
- It is a type of thematic map that uses different shading patterns or colors in predefined areas on a map to represent geographic characteristics in relation to a variable or metric.
- It can be plotted on a world map, U.S. map, or divided by different geographic areas such as states, counties, zip codes, etc.
- The image below shows some examples of choropleth maps from [Wikipedia](https://en.wikipedia.org/wiki/Choropleth_map).

![](https://miro.medium.com/v2/resize:fit:602/1*GPlgH42fGIw2Md0DXRTA9w.png)



- In Python, there are several graphing libraries that you can use to plot choropleth maps such as Folium, Plotly, Matplotlib, etc.
- Among those, Folium is a highly specialized geospatial visualization library.
- It is robust and customizable and can plot choropleth maps literally in any geographic area or country with a variety of flavors and designs.
- It is an ideal choice for any beginners who want to get started with geospatial visualization in Python.



- In this tutorial, we will learn the core concepts of Folium and use it to create a choropleth map that visualizes the Covid-19 cases in the U.S. at the county level, with customized tooltips and layer control.
- The choropleth map we are going to create looks like this:

![](https://miro.medium.com/v2/resize:fit:700/1*cvu6NnwOiPzO25B9pYe6mw.png)


### Step 0: Download the Data

Making Choropleth Maps requires two main types of input:

1. A tabular dataset that contains the geographic area identifiers (e.g., country names, county fips code, zip code, etc.) and metrics that we want to visualize in the map (e.g., population, unemployment rate, etc.)
2. A GeoJSON file that contains the geometry information and defines the boundaries of the pre-defined geographic areas (e.g., countries, U.S. counties, zip codes, etc.)

![](https://miro.medium.com/v2/resize:fit:663/1*VK3mRg2TJPjTfBghmsZ-Bg.png)




- Now that we have understood the data input requirements and what a GeoJSON file does, we can go ahead and download the two datasets needed for our tutorial.
- Both of the files are open datasets that are free to download and use.

**Dataset 1**: [United States COVID-19 County Level of Community Transmission](https://data.cdc.gov/Public-Health-Surveillance/United-States-COVID-19-County-Level-of-Community-T/8396-v7yb/data)

![](https://miro.medium.com/v2/resize:fit:700/1*0AggNoVpmrWyayYpXJUkxA.png)

Data Source: [Center for Disease Control and Prevention](https://data.cdc.gov/Public-Health-Surveillance/United-States-COVID-19-County-Level-of-Community-T/8396-v7yb/data)

**Dataset 2**: [GeoJSON file for U.S. Counties](https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/export/?disjunctive.ste_code=&disjunctive.ste_name=&disjunctive.coty_code=&disjunctive.coty_name=&sort=year)

You can go to [public.opendatasoft.com](https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/export/?disjunctive.ste_code=&disjunctive.ste_name=&disjunctive.coty_code=&disjunctive.coty_name=&sort=year), scroll down to the ‘Geographic file formats’ section, and download the GeoJSON file for the U.S. counties.

![](https://miro.medium.com/v2/resize:fit:312/1*LqJj56Rsf5gXYA_axZHivQ.png)




# Import Libraries and Read the Data

- Let’s import all the necessary libraries and read the data into Python.
- Notice that in order to read GeoJSON file, we need to use GeoPandas, an open-source library that is specifically created for handling geospatial data types and makes working with geospatial data in python so much easier.

Next, let’s take a close look at the covid_df data frame.



In [143]:
#Import Libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip

geojson = gpd.read_file(r'./georef-united-states-of-america-county.geojson')
geojson=geojson[['coty_code','geometry']] #only select 'coty_code' (country fips) and 'geometry' columns
covid_df = pd.read_csv('./covid_dataset_compact.csv')

In [94]:
covid_df

,Unnamed: 0,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level
0,441024,Arkansas,Greene County,5055,2021/12/29,423.610,13.65,high
1,441025,California,Sutter County,6101,2021/12/29,154.690,7.63,high
2,441026,Colorado,Hinsdale County,8053,2021/12/29,0.000,NaN,low
3,441027,Georgia,Bartow County,13015,2021/12/29,636.730,32.54,high
4,441028,Georgia,Dougherty County,13095,2021/12/29,114.830,15.07,high
...,...,...,...,...,...,...,...,...
3216,444240,Puerto Rico,Rincón Muni,72117,2021/12/29,856.770,14.84,high
3217,444241,Puerto Rico,Río Grande Muni,72119,2021/12/29,"1,188.960",25.44,high
3218,444242,Puerto Rico,San Juan Muni,72127,2021/12/29,"1,765.790",21.48,high
3219,444243,Puerto Rico,Toa Alta Muni,72135,2021/12/29,"1,299.550",24.07,high


In [144]:
geojson

,coty_code,geometry
0,[29081],"POLYGON ((-94.23224 40.57190, -94.23235 40.565..."
1,[29099],"POLYGON ((-90.63998 38.07655, -90.63989 38.076..."
2,[29145],"POLYGON ((-94.05921 37.04813, -94.05967 37.048..."
3,[29223],"POLYGON ((-90.77955 37.05032, -90.76834 37.050..."
4,[30053],"POLYGON ((-114.72705 49.00051, -114.75952 49.0..."
...,...,...
3230,[27075],"POLYGON ((-90.79519 47.24430, -90.80062 47.249..."
3231,[27145],"POLYGON ((-95.13188 45.41249, -95.13173 45.412..."
3232,[28043],"POLYGON ((-89.78689 33.67673, -89.78255 33.676..."
3233,[28069],"POLYGON ((-88.34789 32.92908, -88.35697 32.928..."


In [145]:
covid_df.head()

,Unnamed: 0,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level
0,441024,Arkansas,Greene County,5055,2021/12/29,423.610,13.65,high
1,441025,California,Sutter County,6101,2021/12/29,154.690,7.63,high
2,441026,Colorado,Hinsdale County,8053,2021/12/29,0.000,NaN,low
3,441027,Georgia,Bartow County,13015,2021/12/29,636.730,32.54,high
4,441028,Georgia,Dougherty County,13095,2021/12/29,114.830,15.07,high


In [62]:
covid_df[(covid_df['report_date'].str.contains('2021/08/16'))]

,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level



- This dataset keeps track of Covid-19 metrics at the U.S. county level for each report_date.
- Since we are creating a static choropleth map, we’ll limit our data frame to the most recent report_date (i.e. 12/29/2021 in our example).

- We also need to do a bit of data wrangling to clean some of the data fields.
- We will change the data type of ‘fips_code’ to string and fill the leading zero if it’s missing.
- We’ll also create two new columns ‘new_cases_7days’ and ‘pct_positive_7days’ that have correct data types and formats and drop the original columns.



In [146]:
#change the data type of 'fips_code' to string and fill the leading zero
covid_df["fips_code"]=covid_df["fips_code"].map(str)
covid_df["fips_code"] = covid_df["fips_code"].str.zfill(5)

#Filter the dataframe to only show records for 2021/12/29
covid_df=covid_df[(covid_df['report_date'] =='2021/12/29')]

#create two new columns with correct data types and formats and drop the original columns.
covid_df['new_cases_7days'] = covid_df['cases_per_100K_7_day_count_change'].str.replace(',', '')
covid_df['new_cases_7days'] = covid_df['new_cases_7days'].replace({'suppressed': np.nan}).astype(float)
covid_df['pct_positive_7days']=covid_df['percent_test_results_reported_positive_last_7_days']/100
covid_df.drop(['cases_per_100K_7_day_count_change', 'percent_test_results_reported_positive_last_7_days',
              'community_transmission_level'], axis=1, inplace=True)

In [133]:
covid_df.head()

,Unnamed: 0,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
0,441024,Arkansas,Greene County,05055,2021/12/29,423.61,0.1365
1,441025,California,Sutter County,06101,2021/12/29,154.69,0.0763
2,441026,Colorado,Hinsdale County,08053,2021/12/29,0.00,NaN
3,441027,Georgia,Bartow County,13015,2021/12/29,636.73,0.3254
4,441028,Georgia,Dougherty County,13095,2021/12/29,114.83,0.1507



- Lastly, let’s join the geojson with covid_df to create our final dataset that’s ready for visualization.
- By joining the two datasets, we ensure that the fips_code in the covid_df data frame matches exactly the coty_code in the GeoJSON file, which is critical later on when using Folium to plot the choropleth map.



__Merging two dataframes__


how{‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’

Type of merge to be performed.

left: use only keys from left frame, similar to a SQL left outer join; preserve key order.

right: use only keys from right frame, similar to a SQL right outer join; preserve key order.

outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

cross: creates the cartesian product from both frames, preserves the order of the left keys.

In [86]:
# df_final = geojson.merge(covid_df, left_on="coty_code", right_on="fips_code", how="inner")
# df_final = df_final[~df_final['geometry'].isna()]

ValueError: ignored

In [125]:
geojson.head()

,Unnamed: 0,coty_code,geometry
0,0,['29081'],"POLYGON ((-94.23224 40.57190099906777, -94.232..."
1,1,['29099'],"POLYGON ((-90.63998 38.076547999083814, -90.63..."
2,2,['29145'],"POLYGON ((-94.059211 37.04812699909245, -94.05..."
3,3,['29223'],"POLYGON ((-90.779553 37.05032399909244, -90.76..."
4,4,['30053'],"POLYGON ((-114.72705 49.00051399906616, -114.7..."


In [119]:
covid_df.dtypes

Unnamed: 0              int64
state_name             object
county_name            object
fips_code              object
report_date            object
new_cases_7days       float64
pct_positive_7days    float64
dtype: object

In [121]:
geojson.dtypes

Unnamed: 0     int64
coty_code     object
geometry      object
dtype: object

In [120]:
covid_df

,Unnamed: 0,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
0,441024,Arkansas,Greene County,05055,2021/12/29,423.61,0.1365
1,441025,California,Sutter County,06101,2021/12/29,154.69,0.0763
2,441026,Colorado,Hinsdale County,08053,2021/12/29,0.00,NaN
3,441027,Georgia,Bartow County,13015,2021/12/29,636.73,0.3254
4,441028,Georgia,Dougherty County,13095,2021/12/29,114.83,0.1507
...,...,...,...,...,...,...,...
3216,444240,Puerto Rico,Rincón Muni,72117,2021/12/29,856.77,0.1484
3217,444241,Puerto Rico,Río Grande Muni,72119,2021/12/29,1188.96,0.2544
3218,444242,Puerto Rico,San Juan Muni,72127,2021/12/29,1765.79,0.2148
3219,444243,Puerto Rico,Toa Alta Muni,72135,2021/12/29,1299.55,0.2407


In [123]:
geojson

,Unnamed: 0,coty_code,geometry
0,0,[,"POLYGON ((-94.23224 40.57190099906777, -94.232..."
1,1,[,"POLYGON ((-90.63998 38.076547999083814, -90.63..."
2,2,[,"POLYGON ((-94.059211 37.04812699909245, -94.05..."
3,3,[,"POLYGON ((-90.779553 37.05032399909244, -90.76..."
4,4,[,"POLYGON ((-114.72705 49.00051399906616, -114.7..."
...,...,...,...
3230,3230,[,"POLYGON ((-90.795189 47.24429899905977, -90.80..."
3231,3231,[,"POLYGON ((-95.131879 45.41249499905689, -95.13..."
3232,3232,[,"POLYGON ((-89.78689 33.67672699912893, -89.782..."
3233,3233,[,"POLYGON ((-88.347887 32.92908199913867, -88.35..."


In [147]:
# Assuming 'geojson' is your DataFrame
geojson['coty_code'] = geojson['coty_code'].apply(lambda x: x[0] if x else None)
# geojson['coty_code'] = geojson['coty_code'].apply(lambda x: x.strip('[\'\']') if x else None)
geojson.head()

,coty_code,geometry
0,29081,"POLYGON ((-94.23224 40.57190, -94.23235 40.565..."
1,29099,"POLYGON ((-90.63998 38.07655, -90.63989 38.076..."
2,29145,"POLYGON ((-94.05921 37.04813, -94.05967 37.048..."
3,29223,"POLYGON ((-90.77955 37.05032, -90.76834 37.050..."
4,30053,"POLYGON ((-114.72705 49.00051, -114.75952 49.0..."


In [148]:
# Convert data types to string for consistent matching
geojson['coty_code'] = geojson['coty_code'].astype(str)
covid_df['fips_code'] = covid_df['fips_code'].astype(str)

# # Ensure consistent formatting (e.g., leading zeros)
# geojson['coty_code'] = geojson['coty_code'].str.zfill(5)
# covid_df['fips_code'] = covid_df['fips_code'].str.zfill(5)

# Now you can proceed with the merge
df_final = geojson.merge(covid_df, left_on="coty_code", right_on="fips_code", how="outer")

# If you want to filter out rows with missing geometry or other key data
# df_final = df_final.dropna(subset=['geometry', 'state_name'])
df_final = df_final[~df_final['geometry'].isna()]


df_final


,coty_code,geometry,Unnamed: 0,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
0,29081,"POLYGON ((-94.23224 40.57190, -94.23235 40.565...",442250.0,Missouri,Harrison County,29081,2021/12/29,311.30,0.1066
1,29099,"POLYGON ((-90.63998 38.07655, -90.63989 38.076...",441423.0,Missouri,Jefferson County,29099,2021/12/29,473.61,0.1650
2,29145,"POLYGON ((-94.05921 37.04813, -94.05967 37.048...",441429.0,Missouri,Newton County,29145,2021/12/29,185.45,0.1478
3,29223,"POLYGON ((-90.77955 37.05032, -90.76834 37.050...",443036.0,Missouri,Wayne County,29223,2021/12/29,163.13,0.1923
4,30053,"POLYGON ((-114.72705 49.00051, -114.75952 49.0...",442271.0,Montana,Lincoln County,30053,2021/12/29,90.09,0.0692
...,...,...,...,...,...,...,...,...,...
3230,27075,"POLYGON ((-90.79519 47.24430, -90.80062 47.249...",442972.0,Minnesota,Lake County,27075,2021/12/29,159.76,0.0348
3231,27145,"POLYGON ((-95.13188 45.41249, -95.13173 45.412...",442985.0,Minnesota,Stearns County,27145,2021/12/29,366.29,0.0856
3232,28043,"POLYGON ((-89.78689 33.67673, -89.78255 33.676...",442223.0,Mississippi,Grenada County,28043,2021/12/29,289.05,0.1356
3233,28069,"POLYGON ((-88.34789 32.92908, -88.35697 32.928...",441400.0,Mississippi,Kemper County,28069,2021/12/29,184.77,0.0435


In [149]:
df_final[~df_final.isna()]

,coty_code,geometry,Unnamed: 0,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
0,29081,"POLYGON ((-94.23224 40.57190, -94.23235 40.565...",442250.0,Missouri,Harrison County,29081,2021/12/29,311.30,0.1066
1,29099,"POLYGON ((-90.63998 38.07655, -90.63989 38.076...",441423.0,Missouri,Jefferson County,29099,2021/12/29,473.61,0.1650
2,29145,"POLYGON ((-94.05921 37.04813, -94.05967 37.048...",441429.0,Missouri,Newton County,29145,2021/12/29,185.45,0.1478
3,29223,"POLYGON ((-90.77955 37.05032, -90.76834 37.050...",443036.0,Missouri,Wayne County,29223,2021/12/29,163.13,0.1923
4,30053,"POLYGON ((-114.72705 49.00051, -114.75952 49.0...",442271.0,Montana,Lincoln County,30053,2021/12/29,90.09,0.0692
...,...,...,...,...,...,...,...,...,...
3230,27075,"POLYGON ((-90.79519 47.24430, -90.80062 47.249...",442972.0,Minnesota,Lake County,27075,2021/12/29,159.76,0.0348
3231,27145,"POLYGON ((-95.13188 45.41249, -95.13173 45.412...",442985.0,Minnesota,Stearns County,27145,2021/12/29,366.29,0.0856
3232,28043,"POLYGON ((-89.78689 33.67673, -89.78255 33.676...",442223.0,Mississippi,Grenada County,28043,2021/12/29,289.05,0.1356
3233,28069,"POLYGON ((-88.34789 32.92908, -88.35697 32.928...",441400.0,Mississippi,Kemper County,28069,2021/12/29,184.77,0.0435



# Create a Choropleth Map Using Folium

## Step1: Initiate a base folium map

To create a choropleth map using folium, we need to first initiate a base map by using folium.Map() and then add layers to it. We can pass the starting coordinates to the map by using the location parameter. The starting coordinates we choose here (40,-96) approximately represent the center of the U.S. map.

We can select the desired map tile (e.g. tiles=”Stamen Terrain”) from a list of built-in tilesets — the default one is OpenStreetMap, or simply leave the tile option to ‘None’. We can also set the initial zoom level for the map using the zoom_start parameter.



In [130]:

us_map = folium.Map(location=[40, -96], zoom_start=4,tiles='openstreetmap')
us_map


## Step2: Add Choropleth Map Layer to the Base Map

The base map is initially empty. We can create the choropleth map layer using folium.Choropleth() and add it to the base map using add_to() method. Within the folium.Choropleth() function, there are a few import parameters we need to specify:

![](https://miro.medium.com/v2/resize:fit:700/1*lh5WX14P5vWBQ1bvaVxMYA.png)

Notice that in the code above, instead of using fixed value ranges for color scales, we can create a custom scale using quantile() and tolist() and easily pass our custom scale via the threshold_scale parameter. With just a few lines of code, we have created a basic choropleth map with custom color scales using Folium!

![](https://miro.medium.com/v2/resize:fit:700/1*0TfM_X9OW-_VmAZi7gI9rw.png)



In [150]:

#Create the choropleth map add it to the base map
custom_scale = (df_final['new_cases_7days'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
folium.Choropleth(
            geo_data=geojson,
            data=df_final,
            columns=['fips_code', 'new_cases_7days'],  #Here we tell folium to get the county fips and plot new_cases_7days metric for each county
            key_on='feature.properties.coty_code', #Here we grab the geometries/county boundaries from the geojson file using the key 'coty_code' which is the same as county fips
            threshold_scale=custom_scale, #use the custom scale we created for legend
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='New Cases Past 7 Days (Per 100K Population) ', #title of the legend
            highlight=True,
            line_color='black').add_to(us_map)


In [151]:
us_map.save("index.html") #save to a file

In [73]:
us_map

## Step3: Add Customized Tooltips to the Map

The basic map we have created above looks pretty good! However, currently, when users hover over the map, there is no tooltip shown to display information about each county such as county name, number of Covid cases, positive rate, etc. This would be a nice feature to add to the map in order to make the visualization more interactive and informative.

To add the customized tooltips, we need to use the folium.features.GeoJson() method and the GeoJsonTooltip function to set the ‘fields’ and ‘aliases’ parameters. The fields will be any columns in df_final that we want to display in the hover-over tooltips, and the aliases will be the labels/names we give to those fields.

Notice that we can also pass HTML elements to the ‘aliases’ parameter to format the tooltips. For example, we can use <br> to break the long-form text into two lines as shown in the code below.


![](https://miro.medium.com/v2/resize:fit:700/1*LtUJes9sU7BcVEtk6XA8GA.png)

In [ ]:

#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data=df_final,
                    name='New Cases Past 7 days (Per 100K Population)',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['report_date',
                                'county_name',
                                'state_name',
                                'new_cases_7days',
                                'pct_positive_7days'
                               ],
                        aliases=["Report Date:",
                                 "County Name:",
                                 "State Name:",
                                 "New Cases Past 7 days<br>(Per 100K Population):",
                                 "Percent of Positive Cases<br>Past 7days:"
                                ],
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(us_map)

us_map


# Add Layer Control to Toggle Between Metrics

So far our choropleth map only visualizes one metric — ‘new_cases_7days’ — from the df_final data frame. What if we are also interested in seeing how the ‘pct_positive_7days’ metric varies across different counties?

We can follow the same steps shown in the previous section and create another map for ‘pct_positive_7days’. But is there any way we can show multiple metrics in the same map with a toggle button so that we don’t need to show multiple maps separately?

The answer is YES and the secret weapon to solve the problem is using folium.FeatureGroup() and folium.LayerControl().

The folium.FeatureGroup() method creates a FeatureGroup layer. We can put things in it and handle them as a single layer. We can create multiple FeatureGroup layers, each of which represents a metric we want to show on the map. We can then add a LayerControl to tick/untick a FeatureGroup layer and toggle between different metrics.

Let’s see how we can achieve this functionality through the code below:

**Step1:** We initiate a base map and create two FeatureGroup layers called fg1 and fg2 and add them to the base map. We give a name to each featureGroup layer which will be displayed in the LayerControl. The ‘overlay’ parameter allows you to either make the layer an overlay (ticked with a check box in LayerControls) or a base layer (ticked with a radio button).


In [ ]:

#Create two FeatureGroup layers
us_map = folium.Map(location=[40, -96], zoom_start=4,tiles=None,overlay=False)
fg1 = folium.FeatureGroup(name='New Covid-19 Cases Past 7 Days',overlay=False).add_to(us_map)
fg2 = folium.FeatureGroup(name='Percent of Positive Cases Past 7 Days',overlay=False).add_to(us_map)



**Step2:** We add the first choropleth map layer (for ‘new_cases_7days’ metric) to fg1:




In [ ]:

#Add the first choropleth map layer to fg1
custom_scale1 = (df_final['new_cases_7days'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
New_cases=folium.Choropleth(
            geo_data=r'...\georef-united-states-of-america-county.geojson',
            data=df_final,
            columns=['fips_code', 'new_cases_7days'],
            key_on='feature.properties.coty_code',
            threshold_scale=custom_scale1, #use the custom scale we created for legend
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='New Cases Past 7 Days (Per 100K Population) ',
            highlight=True,
            overlay=False,
            line_color='black').geojson.add_to(fg1)

#Add customized tooltips to the map
folium.features.GeoJson(
                    data=df_final,
                    name='New Cases Past 7 days (Per 100K Population)',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['report_date',
                                'county_name',
                                'state_name',
                                'new_cases_7days',
                                'pct_positive_7days',
                               ],
                        aliases=["Report Date:",
                                 "County Name:",
                                 "State Name:",
                                 "New Cases Past 7 days<br>(Per 100K Population):",
                                 "Percent of Positive Cases<br>Past 7days:",
                                ],
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(New_cases)


**Step 3:** We add the second choropleth map layer (for the ‘pct_positive_7days’ metric) to fg2. The code structure is the same as in step 2 except for a few changes related to the ‘pct_postive_7days’ metric.



In [ ]:
#Add the second choropleth map layer to fg2
custom_scale2 = (df_final['pct_positive_7days'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
Pct_cases=folium.Choropleth(
            geo_data=r'...\georef-united-states-of-america-county.geojson',
            data=df_final,
            columns=['fips_code', 'pct_positive_7days'],  #Here we tell folium to get the county fips and plot the 'pct_positive_7days' metric for each county
            key_on='feature.properties.coty_code', #Here we grab the geometries/county boundaries from the geojson file using the key 'coty_code' which is the same as fips_code
            threshold_scale=custom_scale2, #use the custom scale we created for legend
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Percent of Positive Cases Past 7 Days ',
            highlight=True,
            overlay=False,
            line_color='black').geojson.add_to(fg2)

#Add customized tooltips to the map
folium.features.GeoJson(
                    data=df_final,
                    name='Percent of Positive Cases Past 7 Days',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['report_date',
                                'county_name',
                                'state_name',
                                'new_cases_7days',
                                'pct_positive_7days',
                               ],
                        aliases=["Report Date:",
                                 "County Name:",
                                 "State Name:",
                                 "New Cases Past 7 days<br>(Per 100K Population):",
                                 "Percent of Positive Cases<br>Past 7days:",
                                ],
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(Pct_cases)



**Step 4:** We add LayerControl to the map. Notice that in the code I also added a tile layer to choose between dark mode and light mode. We can also save the interactive map to an HTML file.


![](https://miro.medium.com/v2/resize:fit:700/1*6et5BAFSGiC1ujM6zJHfXw.gif)



In [ ]:
#Add layer control to the map
folium.TileLayer('cartodbdark_matter',overlay=True,name="View in Dark Mode").add_to(us_map)
folium.TileLayer('cartodbpositron',overlay=True,name="Viw in Light Mode").add_to(us_map)
folium.LayerControl(collapsed=False).add_to(us_map)
us_map


In [40]:
us_map.save("index.html") #save to a file